In [286]:
import numpy as np
import pandas as pd
from matplotlib.pyplot import grid, figure, plot, savefig
import matplotlib.pyplot as plt
import os
import time


In [335]:

lat = 42.46386370313852 #shitot ot ecvat   MARKS   TPLK
lon = 78.52905690246126# dolg merid-i po ekvat
ls_files =[]
test = []
# Указываем папку в которой производить поиск
path='./'
for rootdir, dirs, files in os.walk(path):
    for file in files:       
        if((file.split('.')[-1])=='dat'):
            ls_files.append(file)
            #print (os.path.join(rootdir, file))
            

cnt_file = len(ls_files)
for i in range(cnt_file):
    t=[]
    #test.append(i)
    file1 = open (ls_files[i])
    #print(file1)
    allLines = file1.readlines()
    file1linecount = len(allLines)
    file1.seek(0) # peremestit ukozatel v nachal

    for ln in range(file1linecount) :#3700 strok все строки
    #for ln in range(file1) :#3700 strok все строки
        line  = file1.readline()
        if ln > 10 :
            param=line.strip() #str
            param = param.split()
           # print(param)
            t.append(param)
        #np.array(param)
    test.append(t)
arr = np.array(test)
arr = arr.astype(float)
G5 = pd.DataFrame(arr[0],  columns=['tsn', 'hour', 'el', 'az','phase','code','valid'])
G5['Cr']=  G5['code']-G5['phase'] 
G5['Coef']= ((1/G5['Cr'].count())*G5['Cr'].sum())
G5['sTEC']= G5['phase']+G5['Coef']
G5['vTEC'] =  G5['sTEC']*  (np.cos  (np.arcsin(  (6400*np.cos(G5['el']*3.14/180))  /  (6400+350) )  )     ) #
G5['psi'] = np.pi/2 - np.radians(G5['el'])- np.arcsin(6371 * np.cos( np.radians(G5['el']) )/(6371+350) ) #1.209
G5['lat']=  np.degrees(np.arcsin( np.sin(np.radians(lat)) * np.cos(G5['psi']) + np.cos(np.radians(lat))*np.sin(G5['psi']) *np.cos(np.radians(G5['az'])) ))
G5['lon'] = np.degrees(np.radians(lon)+ np.arcsin( np.sin( G5['psi'] ) * np.sin(np.radians( G5['az'] ) ) * (1/np.cos( np.radians( G5['lat'] )) ) )  ) 
G5['lat^2']= G5['lat']**2
G5['lon^2']= G5['lon']**2
G5['S'] = np.cos(  np.arcsin( 6371* np.sin(0.97*np.radians(90-G5['el']))/(6371+350) )    )**-1 # factor naklona
print('ddddddd')
print(G5['S'])
G5.drop(['psi','valid','az','el','phase','code','Cr','Coef'], axis=1, inplace=True) 
G5 = G5[0:30]
G5['t']=G5['hour']-G5['hour'].iloc[0]# пока е0 минус t каждое
G5['EnBg']=G5['hour'].iloc[-1]-G5['hour'].iloc[0]# пока е0 минус t каждое
print(G5['EnBg'])

G5['t^2'] = G5['t']**2
G5['w'] = 1/G5['S'] *(  1+ ((2* G5['t'])/0.5)**2     )**-1 # ves =  delt t / interval 
G5['(w)']=G5['w'].sum()
G5['(w*S)']=(G5['w']*G5['S']).sum()
G5['(w*S*lat)']=(G5['w']*G5['S']*G5['lat']).sum()
G5['(w*S*lat^2)']=(G5['w']*G5['S']*G5['lat^2']).sum()
G5['(w*S*lon)']=(G5['w']*G5['S']*G5['lon']).sum()
G5['(w*S*lon^2)']=(G5['w']*G5['S']*G5['lon^2']).sum()
G5['(w*S*t)']=  (G5['w']*G5['S']*G5['t']).sum()
G5['(w*S*t^2)']=(G5['w']*G5['S']*G5['t^2']).sum()
G5['(w*I)']=G5['w']*G5['vTEC'].sum()

G5['w*S^2']=        (G5['w']*G5['S']**2)
G5['w*S^2*lat'] =   (G5['w']*G5['S']**2 * G5['lat'])
G5['w*S^2*lat^2']=  (G5['w']*G5['S']**2 * G5['lat']**2)
G5['w*S^2*lat^3']=  (G5['w']*G5['S']**2 * G5['lat']**3)
G5['w*S^2*lat^4']=  (G5['w']*G5['S']**2 * G5['lat']**4)
G5['w*S^2*lon'] =   (G5['w']*G5['S']**2 * G5['lon'])
G5['w*S^2*lon^2'] = (G5['w']*G5['S']**2 * G5['lon']**2)
G5['w*S^2*lon^3'] = (G5['w']*G5['S']**2 * G5['lon']**3)
G5['w*S^2*lon^4'] = (G5['w']*G5['S']**2 * G5['lon']**4)

G5['w*S^2*t']=          (G5['w'] * G5['S']**2 * G5['t'])
G5['w*S^2*t^2']=        (G5['w'] * G5['S']**2 * G5['t']**2) 
G5['w*S^2*t^3']=        (G5['w'] * G5['S']**2 * G5['t']**3) 
G5['w*S^2*t^4']=        (G5['w'] * G5['S']**2 * G5['t']**4) 
G5['w*S^2*lat*lon']=    (G5['w'] * G5['S']**2 * G5['lat'] * G5['lon'])
G5['w*S^2*lat^2*lon']=  (G5['w'] * G5['S']**2 * G5['lat']**2 * G5['lon'])
G5['w*S^2*lat*lon^2']=  (G5['w'] * G5['S']**2 * G5['lat'] * G5['lon']**2)
G5['w*S^2*lat^2*lon^2']=(G5['w'] * G5['S']**2 * G5['lat']**2 * G5['lon']**2)
G5['w*S^2*lat*t']=      (G5['w'] * G5['S']**2 * G5['lat'] * G5['t'])
G5['w*S^2*lat^2*t']=    (G5['w'] * G5['S']**2 * G5['lat']**2 * G5['t'])
G5['w*S^2*lat*t^2']=    (G5['w'] * G5['S']**2 * G5['lat'] * G5['t']**2)
G5['w*S^2*lat^2*t^2']=  (G5['w'] * G5['S']**2 * G5['lat']**2 * G5['t']**2)
G5['w*S^2*lon*t']=      (G5['w'] * G5['S'] *    G5 [ 'lon'] * G5[' t'])
G5['w*S^2*lon^2*t']=    (G5['w'] * G5['S']**2 * G5['lon']**2 * G5['t'])
G5['w*S^2*lon*t^2']=    (G5['w'] * G5['S']**2 * G5['lon'] * G5['t']**2)
G5['w*S^2*lon^2*t^2']=  (G5['w'] * G5['S']**2 * G5['lon']**2 * G5['t']**2)
G5['w*S*I'] =           (G5['w'] * G5['S'] * G5 [ 'vTEC'])
G5['w*S*I*lat'] =       (G5['w'] * G5['S'] * G5 [ 'vTEC'] *G5['lat'])
G5['w*S*I*lat^2'] =     (G5['w'] * G5['S'] * G5 [ 'vTEC'] * G5['lat']**2)
G5['w*S*I*lon'] =       (G5['w'] * G5['S'] * G5 [ 'vTEC'] * G5['lon'])
G5['w*S*I*lon^2'] =     (G5['w'] * G5['S'] * G5 [ 'vTEC'] * G5['lon']**2)
G5['w*S*I*t'] =         (G5['w'] * G5['S'] * G5 [ 'vTEC'] * G5['t'])
G5['w*S*I*t^2'] =       (G5['w'] * G5['S'] * G5 [ 'vTEC'] * G5['t']**2)

G10 = pd.DataFrame(arr[1],  columns=['tsn', 'hour', 'el', 'az','phase','code','valid'])
G10['Cr']=  G10['code']-G10['phase'] 
G10['Coef']= ((1/G10['Cr'].count())*G10['Cr'].sum())
G10['sTEC']= G10['phase']+G10['Coef']
G10['vTEC'] =  G10['sTEC']*  (np.cos  (np.arcsin(  (6400*np.cos(G10['el']*3.14/180))  /  (6400+350) )  )     ) #
G10['psi'] = np.pi/2 - np.radians(G10['el'])- np.arcsin(6371 * np.cos( np.radians(G10['el']) )/(6371+350) ) #1.209
G10['lat']=  np.degrees(np.arcsin( np.sin(np.radians(lat)) * np.cos(G10['psi']) + np.cos(np.radians(lat))*np.sin(G10['psi']) *np.cos(np.radians(G10['az'])) ))
G10['lon'] = np.degrees(np.radians(lon)+ np.arcsin( np.sin( G10['psi'] ) * np.sin(np.radians( G10['az'] ) ) * (1/np.cos( np.radians( G10['lat'] )) ) )  ) 
G10['lat^2']= G10['lat']**2
G10['lon^2']= G10['lon']**2
G10['S'] = np.cos(  np.arcsin( 6371* np.sin(0.97*np.radians(90-G10['el']))/(6371+350) )    )**-1 # factor naklona
G10.drop(['psi','valid','az','el','phase','code','Cr','Coef'], axis=1, inplace=True) 
G10 = G10[0:30]
G10['t']=G10['hour']-G10['hour'].iloc[0]# пока е0 минус t каждое
G10['t^2'] = G10['t']**2
G10['w'] = 1/G10['S'] *(  1+ ((2* G10['t'])/0.5)**2     )**-1 # ves =  delt t / interval
G10['(w)']=G10['w'].sum()
G10['(w*S)']=(G10['w']*G10['S']).sum()
G10['(w*S*lat)']=(G10['w']*G10['S']*G10['lat']).sum()
G10['(w*S*lat^2)']=(G10['w']*G10['S']*G10['lat^2']).sum()
G10['(w*S*lon)']=(G10['w']*G10['S']*G10['lon']).sum()
G10['(w*S*lon^2)']=(G10['w']*G10['S']*G10['lon^2']).sum()
G10['(w*S*t)']=  (G10['w']*G10['S']*G10['t']).sum()
G10['(w*S*t^2)']=(G10['w']*G10['S']*G10['t^2']).sum()
G10['(w*I)']=G10['w']*G10['vTEC'].sum()
G10['w*S^2']=(G10['w']*G10['S']**2)
G10['w*S^2*lat'] = (G10['w']*G10['S']**2*G10['lat'])
G10['w*S^2*lat^2']=(G10['w']*G10['S']**2*G10['lat']**2)
G10['w*S^2*lat^3']=(G10['w']*G10['S']**2*G10['lat']**3)
G10['w*S^2*lat^4']=(G10['w']*G10['S']**2*G10['lat']**4)
G10['w*S^2*lon'] = (G10['w']*G10['S']**2*G10['lon'])
G10['w*S^2*lon^2'] = (G10['w']*G10['S']**2*G10['lon']**2)
G10['w*S^2*lon^3'] = (G10['w']*G10['S']**2*G10['lon']**3)
G10['w*S^2*lon^4'] = (G10['w']*G10['S']**2*G10['lon']**4)
G10['w*S^2*lon']=(G10['w']*G10['S']**2*G10['lon'])
G10['w*S^2*t']=(G10['w']*G10['S']**2*G10['t'])
G10['w*S^2*t^2']=(G10['w']*G10['S']**2*G10['t']**2)
G10['w*S^2*t^3']=(G10['w']*G10['S']**2*G10['t']**3)
G10['w*S^2*t^4']=(G10['w']*G10['S']**2*G10['t']**4)
G10['w*S^2*lat*lon']=(G10['w']*G10['S']**2*G10['lat']*G10['lon'])
G10['w*S^2*lat^2*lon']=(G10['w']*G10['S']**2*G10['lat']**2*G10['lon'])
G10['w*S^2*lat*lon^2']=(G10['w']*G10['S']**2*G10['lat']*G10['lon']**2)
G10['w*S^2*lat^2*lon^2']=(G10['w']*G10['S']**2*G10['lat']**2*G10['lon']**2)
G10['w*S^2*lat*t']=(G10['w']*G10['S']**2*G10['lat']*G10['t'])
G10['w*S^2*lat^2*t']=(G10['w']*G10['S']**2*G10['lat']**2*G10['t'])
G10['w*S^2*lat*t^2']=(G10['w']*G10['S']**2*G10['lat']*G10['t']**2)
G10['w*S^2*lat^2*t^2']=(G10['w']*G10['S']**2*G10['lat']**2*G10['t']**2)
G10['w*S^2*lon*t']=  (G10['w']*G10['S']*G10['lon']*G10['t'])
G10['w*S^2*lon^2*t']=  (G10['w']*G10['S']**2*G10['lon']**2*G10['t'])
G10['w*S^2*lon*t^2']=  (G10['w']*G10['S']**2*G10['lon']*G10['t']**2)
G10['w*S^2*lon^2*t^2']=(G10['w']*G10['S']**2*G10['lon']**2*G10['t']**2)
G10['w*S*I'] = (G10['w']*G10['S']*G10['vTEC'])
G10['w*S*I*lat'] = (G10['w']*G10['S']*G10['vTEC']*G10['lat'])
G10['w*S*I*lat^2'] = (G10['w']*G10['S']*G10['vTEC']*G10['lat']**2)
G10['w*S*I*lon'] = (G10['w']*G10['S']*G10['vTEC']*  G10['lon'])
G10['w*S*I*lon^2'] = (G10['w']*G10['S']*G10['vTEC']*G10['lon']**2)
G10['w*S*I*t'] = (G10['w']*G10['S']*G10['vTEC']*  G10['t'])
G10['w*S*I*t^2'] = (G10['w']*G10['S']*G10['vTEC']*G10['t']**2)

G13 = pd.DataFrame(arr[3],  columns=['tsn', 'hour', 'el', 'az','phase','code','valid'])
G13['Cr']=  G13['code']-G13['phase'] 
G13['Coef']= ((1/G13['Cr'].count())*G13['Cr'].sum())
G13['sTEC']= G13['phase']+G13['Coef']
G13['vTEC'] =  G13['sTEC']*  (np.cos  (np.arcsin(  (6400*np.cos(G13['el']*3.14/180))  /  (6400+350) )  )     ) #
G13['psi'] = np.pi/2 - np.radians(G13['el'])- np.arcsin(6371 * np.cos( np.radians(G13['el']) )/(6371+350) ) #1.209
G13['lat']=  np.degrees(np.arcsin( np.sin(np.radians(lat)) * np.cos(G13['psi']) + np.cos(np.radians(lat))*np.sin(G13['psi']) *np.cos(np.radians(G13['az'])) ))
G13['lon'] = np.degrees(np.radians(lon)+ np.arcsin( np.sin( G13['psi'] ) * np.sin(np.radians( G13['az'] ) ) * (1/np.cos( np.radians( G13['lat'] )) ) )  ) 
G13['lat^2']= G13['lat']**2
G13['lon^2']= G13['lon']**2
G13['S'] = np.cos(  np.arcsin( 6371* np.sin(0.97*np.radians(90-G13['el']))/(6371+350) )    )**-1 # factor naklona
G13.drop(['psi','valid','az','el','phase','code','Cr','Coef'], axis=1, inplace=True) 
G13 = G13[0:30]
G13['t']=G13['hour']-G13['hour'].iloc[0]# пока е0 минус t каждое
G13['t^2'] = G13['t']**2
G13['w'] = 1/G13['S'] *(  1+ ((2* G13['t'])/0.5)**2     )**-1 # ves =  delt t / interval 
G13['(w)']=G13['w'].sum()
G13['(w*S)']=(G13['w']*G13['S']).sum()
G13['(w*S*lat)']=(G13['w']*G13['S']*G13['lat']).sum()
G13['(w*S*lat^2)']=(G13['w']*G13['S']*G13['lat^2']).sum()
G13['(w*S*lon)']=(G13['w']*G13['S']*G13['lon']).sum()
G13['(w*S*lon^2)']=(G13['w']*G13['S']*G13['lon^2']).sum()
G13['(w*S*t)']=  (G13['w']*G13['S']*G13['t']).sum()
G13['(w*S*t^2)']=(G13['w']*G13['S']*G13['t^2']).sum()
G13['(w*I)']=G13['w']*G13['vTEC'].sum()
G13['w*S^2']=(G13['w']*G13['S']**2)
G13['w*S^2*lat'] = (G13['w']*G13['S']**2*G13['lat'])
G13['w*S^2*lat^2']=(G13['w']*G13['S']**2*G13['lat']**2)
G13['w*S^2*lat^3']=(G13['w']*G13['S']**2*G13['lat']**3)
G13['w*S^2*lat^4']=(G13['w']*G13['S']**2*G13['lat']**4)
G13['w*S^2*lon'] = (G13['w']*G13['S']**2*G13['lon'])
G13['w*S^2*lon^2'] = (G13['w']*G13['S']**2*G13['lon']**2)
G13['w*S^2*lon^3'] = (G13['w']*G13['S']**2*G13['lon']**3)
G13['w*S^2*lon^4'] = (G13['w']*G13['S']**2*G13['lon']**4)
G13['w*S^2*lon']=(G13['w']*G13['S']**2*G13['lon'])
G13['w*S^2*t']=(G13['w']*G13['S']**2*G13['t'])
G13['w*S^2*t^2']=(G13['w']*G13['S']**2*G13['t']**2)
G13['w*S^2*t^3']=(G13['w']*G13['S']**2*G13['t']**3)
G13['w*S^2*t^4']=(G13['w']*G13['S']**2*G13['t']**4)
G13['w*S^2*lat*lon']=(G13['w']*G13['S']**2*G13['lat']*G13['lon'])
G13['w*S^2*lat^2*lon']=(G13['w']*G13['S']**2*G13['lat']**2*G13['lon'])
G13['w*S^2*lat*lon^2']=(G13['w']*G13['S']**2*G13['lat']*G13['lon']**2)
G13['w*S^2*lat^2*lon^2']=(G13['w']*G13['S']**2*G13['lat']**2*G13['lon']**2)
G13['w*S^2*lat*t']=(G13['w']*G13['S']**2*G13['lat']*G13['t'])
G13['w*S^2*lat^2*t']=(G13['w']*G13['S']**2*G13['lat']**2*G13['t'])
G13['w*S^2*lat*t^2']=(G13['w']*G13['S']**2*G13['lat']*G13['t']**2)
G13['w*S^2*lat^2*t^2']=(G13['w']*G13['S']**2*G13['lat']**2*G13['t']**2)
G13['w*S^2*lon*t']=  (G13['w']*G13['S']*G13['lon']*G13['t'])
G13['w*S^2*lon^2*t']=  (G13['w']*G13['S']**2*G13['lon']**2*G13['t'])
G13['w*S^2*lon*t^2']=  (G13['w']*G13['S']**2*G13['lon']*G13['t']**2)
G13['w*S^2*lon^2*t^2']=(G13['w']*G13['S']**2*G13['lon']**2*G13['t']**2)
G13['w*S*I'] = (G13['w']*G13['S']*G13['vTEC'])
G13['w*S*I*lat'] = (G13['w']*G13['S']*G13['vTEC']*G13['lat'])
G13['w*S*I*lat^2'] = (G13['w']*G13['S']*G13['vTEC']*G13['lat']**2)
G13['w*S*I*lon'] = (G13['w']*G13['S']*G13['vTEC']*  G13['lon'])
G13['w*S*I*lon^2'] = (G13['w']*G13['S']*G13['vTEC']*G13['lon']**2)
G13['w*S*I*t'] = (G13['w']*G13['S']*G13['vTEC']*  G13['t'])
G13['w*S*I*t^2'] = (G13['w']*G13['S']*G13['vTEC']*G13['t']**2)

G15 = pd.DataFrame(arr[2],  columns=['tsn', 'hour', 'el', 'az','phase','code','valid'])
G15['Cr']=  G15['code']-G15['phase'] 
G15['Coef']= ((1/G15['Cr'].count())*G15['Cr'].sum())
G15['sTEC']= G15['phase']+G15['Coef']
G15['vTEC'] =  G15['sTEC']*  (np.cos  (np.arcsin(  (6400*np.cos(G15['el']*3.14/180))  /  (6400+350) )  )     ) #
G15['psi'] = np.pi/2 - np.radians(G15['el'])- np.arcsin(6371 * np.cos( np.radians(G15['el']) )/(6371+350) ) #1.209
G15['lat']=  np.degrees(np.arcsin( np.sin(np.radians(lat)) * np.cos(G15['psi']) + np.cos(np.radians(lat))*np.sin(G15['psi']) *np.cos(np.radians(G15['az'])) ))
G15['lon'] = np.degrees(np.radians(lon)+ np.arcsin( np.sin( G15['psi'] ) * np.sin(np.radians( G15['az'] ) ) * (1/np.cos( np.radians( G15['lat'] )) ) )  ) 
G15['lat^2']= G15['lat']**2
G15['lon^2']= G15['lon']**2
G15['S'] = np.cos(  np.arcsin( 6371* np.sin(0.97*np.radians(90-G15['el']))/(6371+350) )    )**-1 # factor naklona
G15.drop(['psi','valid','az','el','phase','code','Cr','Coef'], axis=1, inplace=True) 
G15 = G15[0:30]
G15['t']=G15['hour']-G15['hour'].iloc[0]# пока е0 минус t каждое
G15['t^2'] = G15['t']**2
G15['w'] = 1/G15['S'] *(  1+ ((2* G15['t'])/0.5)**2     )**-1 # ves =  delt t / interval
G15['(w)']=G15['w'].sum()
G15['(w*S)']=(G15['w']*G15['S']).sum()
G15['(w*S*lat)']=(G15['w']*G15['S']*G15['lat']).sum()
G15['(w*S*lat^2)']=(G15['w']*G15['S']*G15['lat^2']).sum()
G15['(w*S*lon)']=(G15['w']*G15['S']*G15['lon']).sum()
G15['(w*S*lon^2)']=(G15['w']*G15['S']*G15['lon^2']).sum()
G15['(w*S*t)']=  (G15['w']*G15['S']*G15['t']).sum()
G15['(w*S*t^2)']=(G15['w']*G15['S']*G15['t^2']).sum()
G15['(w*I)']=G15['w']*G15['vTEC'].sum()
G15['w*S^2']=(G15['w']*G15['S']**2)
G15['w*S^2*lat'] = (G15['w']*G15['S']**2*G15['lat'])
G15['w*S^2*lat^2']=(G15['w']*G15['S']**2*G15['lat']**2)
G15['w*S^2*lat^3']=(G15['w']*G15['S']**2*G15['lat']**3)
G15['w*S^2*lat^4']=(G15['w']*G15['S']**2*G15['lat']**4)
G15['w*S^2*lon'] = (G15['w']*G15['S']**2*G15['lon'])
G15['w*S^2*lon^2'] = (G15['w']*G15['S']**2*G15['lon']**2)
G15['w*S^2*lon^3'] = (G15['w']*G15['S']**2*G15['lon']**3)
G15['w*S^2*lon^4'] = (G15['w']*G15['S']**2*G15['lon']**4)
G15['w*S^2*lon']=(G15['w']*G15['S']**2*G15['lon'])
G15['w*S^2*t']=(G15['w']*G15['S']**2*G15['t'])
G15['w*S^2*t^2']=(G15['w']*G15['S']**2*G15['t']**2)
G15['w*S^2*t^3']=(G15['w']*G15['S']**2*G15['t']**3)
G15['w*S^2*t^4']=(G15['w']*G15['S']**2*G15['t']**4)
G15['w*S^2*lat*lon']=(G15['w']*G15['S']**2*G15['lat']*G15['lon'])
G15['w*S^2*lat^2*lon']=(G15['w']*G15['S']**2*G15['lat']**2*G15['lon'])
G15['w*S^2*lat*lon^2']=(G15['w']*G15['S']**2*G15['lat']*G15['lon']**2)
G15['w*S^2*lat^2*lon^2']=(G15['w']*G15['S']**2*G15['lat']**2*G15['lon']**2)
G15['w*S^2*lat*t']=(G15['w']*G15['S']**2*G15['lat']*G15['t'])
G15['w*S^2*lat^2*t']=(G15['w']*G15['S']**2*G15['lat']**2*G15['t'])
G15['w*S^2*lat*t^2']=(G15['w']*G15['S']**2*G15['lat']*G15['t']**2)
G15['w*S^2*lat^2*t^2']=(G15['w']*G15['S']**2*G15['lat']**2*G15['t']**2)
G15['w*S^2*lon*t']=  (G15['w']*G15['S']*G15['lon']*G15['t'])
G15['w*S^2*lon^2*t']=  (G15['w']*G15['S']**2*G15['lon']**2*G15['t'])
G15['w*S^2*lon*t^2']=  (G15['w']*G15['S']**2*G15['lon']*G15['t']**2)
G15['w*S^2*lon^2*t^2']=(G15['w']*G15['S']**2*G15['lon']**2*G15['t']**2)
G15['w*S*I'] = (G15['w']*G15['S']*G15['vTEC'])
G15['w*S*I*lat'] = (G15['w']*G15['S']*G15['vTEC']*G15['lat'])
G15['w*S*I*lat^2'] = (G15['w']*G15['S']*G15['vTEC']*G15['lat']**2)
G15['w*S*I*lon'] = (G15['w']*G15['S']*G15['vTEC']*  G15['lon'])
G15['w*S*I*lon^2'] = (G15['w']*G15['S']*G15['vTEC']*G15['lon']**2)
G15['w*S*I*t'] = (G15['w']*G15['S']*G15['vTEC']*  G15['t'])
G15['w*S*I*t^2'] = (G15['w']*G15['S']*G15['vTEC']*G15['t']**2)

G18 = pd.DataFrame(arr[4],  columns=['tsn', 'hour', 'el', 'az','phase','code','valid'])
G18['Cr']=  G18['code']-G18['phase'] 
G18['Coef']= ((1/G18['Cr'].count())*G18['Cr'].sum())
G18['sTEC']= G18['phase']+G18['Coef']
G18['vTEC'] =  G18['sTEC']*  (np.cos  (np.arcsin(  (6400*np.cos(G18['el']*3.14/180))  /  (6400+350) )  )     ) #
G18['psi'] = np.pi/2 - np.radians(G18['el'])- np.arcsin(6371 * np.cos( np.radians(G18['el']) )/(6371+350) ) #1.209
G18['lat']=  np.degrees(np.arcsin( np.sin(np.radians(lat)) * np.cos(G18['psi']) + np.cos(np.radians(lat))*np.sin(G18['psi']) *np.cos(np.radians(G18['az'])) ))
G18['lon'] = np.degrees(np.radians(lon)+ np.arcsin( np.sin( G18['psi'] ) * np.sin(np.radians( G18['az'] ) ) * (1/np.cos( np.radians( G18['lat'] )) ) )  ) 
G18['lat^2']= G18['lat']**2
G18['lon^2']= G18['lon']**2
G18['S'] = np.cos(  np.arcsin( 6371* np.sin(0.97*np.radians(90-G18['el']))/(6371+350) )    )**-1 # factor naklona
G18.drop(['psi','valid','az','el','phase','code','Cr','Coef'], axis=1, inplace=True) 
G18 = G18[0:30]
G18['t']=G18['hour']-G18['hour'].iloc[0]# пока е0 минус t каждое
G18['t^2'] = G18['t']**2
G18['w'] = 1/G18['S'] *(  1+ ((2* G18['t'])/0.5)**2     )**-1 # ves =  delt t / interval
G18['(w)']=G18['w'].sum()
G18['(w*S)']=(G18['w']*G18['S']).sum()
G18['(w*S*lat)']=(G18['w']*G18['S']*G18['lat']).sum()
G18['(w*S*lat^2)']=(G18['w']*G18['S']*G18['lat^2']).sum()
G18['(w*S*lon)']=(G18['w']*G18['S']*G18['lon']).sum()
G18['(w*S*lon^2)']=(G18['w']*G18['S']*G18['lon^2']).sum()
G18['(w*S*t)']=  (G18['w']*G18['S']*G18['t']).sum()
G18['(w*S*t^2)']=  (G18['w']*G18['S']*G18['t^2']).sum()
G18['(w*I)']=G18['w']*G18['vTEC'].sum()

G18['w*S^2']=(G18['w']*G18['S']**2)
G18['w*S^2*lat'] = (G18['w']*G18['S']**2*G18['lat'])
G18['w*S^2*lat^2']=(G18['w']*G18['S']**2*G18['lat']**2)
G18['w*S^2*lat^3']=(G18['w']*G18['S']**2*G18['lat']**3)
G18['w*S^2*lat^4']=(G18['w']*G18['S']**2*G18['lat']**4)
G18['w*S^2*lon'] = (G18['w']*G18['S']**2*G18['lon'])
G18['w*S^2*lon^2'] = (G18['w']*G18['S']**2*G18['lon']**2)
G18['w*S^2*lon^3'] = (G18['w']*G18['S']**2*G18['lon']**3)
G18['w*S^2*lon^4'] = (G18['w']*G18['S']**2*G18['lon']**4)
G18['w*S^2*lon']=(G18['w']*G18['S']**2*G18['lon'])
G18['w*S^2*t']=(G18['w']*G18['S']**2*G18['t'])
G18['w*S^2*t^2']=(G18['w']*G18['S']**2*G18['t']**2)
G18['w*S^2*t^3']=(G18['w']*G18['S']**2*G18['t']**3)
G18['w*S^2*t^4']=(G18['w']*G18['S']**2*G18['t']**4)
G18['w*S^2*lat*lon']=(G18['w']*G18['S']**2*G18['lat']*G18['lon'])
G18['w*S^2*lat^2*lon']=(G18['w']*G18['S']**2*G18['lat']**2*G18['lon'])
G18['w*S^2*lat*lon^2']=(G18['w']*G18['S']**2*G18['lat']*G18['lon']**2)
G18['w*S^2*lat^2*lon^2']=(G18['w']*G18['S']**2*G18['lat']**2*G18['lon']**2)
G18['w*S^2*lat*t']=(G18['w']*G18['S']**2*G18['lat']*G18['t'])
G18['w*S^2*lat^2*t']=(G18['w']*G18['S']**2*G18['lat']**2*G18['t'])
G18['w*S^2*lat*t^2']=(G18['w']*G18['S']**2*G18['lat']*G18['t']**2)
G18['w*S^2*lat^2*t^2']=(G18['w']*G18['S']**2*G18['lat']**2*G18['t']**2)
G18['w*S^2*lon*t']=  (G18['w']*G18['S']*G18['lon']*G18['t'])
G18['w*S^2*lon^2*t']=  (G18['w']*G18['S']**2*G18['lon']**2*G18['t'])
G18['w*S^2*lon*t^2']=  (G18['w']*G18['S']**2*G18['lon']*G18['t']**2)
G18['w*S^2*lon^2*t^2']=(G18['w']*G18['S']**2*G18['lon']**2*G18['t']**2)
G18['w*S*I'] = (G18['w']*G18['S']*G18['vTEC'])
G18['w*S*I*lat'] = (G18['w']*G18['S']*G18['vTEC']*G18['lat'])
G18['w*S*I*lat^2'] = (G18['w']*G18['S']*G18['vTEC']*G18['lat']**2)
G18['w*S*I*lon'] = (G18['w']*G18['S']*G18['vTEC']*  G18['lon'])
G18['w*S*I*lon^2'] = (G18['w']*G18['S']*G18['vTEC']*G18['lon']**2)
G18['w*S*I*t'] = (G18['w']*G18['S']*G18['vTEC']*  G18['t'])
G18['w*S*I*t^2'] = (G18['w']*G18['S']*G18['vTEC']*G18['t']**2)
G18.to_csv('G18.csv',sep=';')

G23 = pd.DataFrame(arr[5],  columns=['tsn', 'hour', 'el', 'az','phase','code','valid'])
G23['Cr']=  G23['code']-G23['phase'] 
G23['Coef']= ((1/G23['Cr'].count())*G23['Cr'].sum())
G23['sTEC']= G23['phase']+G23['Coef']
G23['vTEC'] =  G23['sTEC']*  (np.cos  (np.arcsin(  (6400*np.cos(G23['el']*3.14/180))  /  (6400+350) )  )     ) #
G23['psi'] = np.pi/2 - np.radians(G23['el'])- np.arcsin(6371 * np.cos( np.radians(G23['el']) )/(6371+350) ) #1.209
G23['lat']=  np.degrees(np.arcsin( np.sin(np.radians(lat)) * np.cos(G23['psi']) + np.cos(np.radians(lat))*np.sin(G23['psi']) *np.cos(np.radians(G23['az'])) ))
G23['lon'] = np.degrees(np.radians(lon)+ np.arcsin( np.sin( G23['psi'] ) * np.sin(np.radians( G23['az'] ) ) * (1/np.cos( np.radians( G23['lat'] )) ) )  ) 
G23['lat^2']= G23['lat']**2
G23['lon^2']= G23['lon']**2
G23['S'] = np.cos(  np.arcsin( 6371* np.sin(0.97*np.radians(90-G23['el']))/(6371+350) )    )**-1 # factor naklona
G23.drop(['psi','valid','az','el','phase','code','Cr','Coef'], axis=1, inplace=True) 
G23 = G23[0:30]
G23['t']=G23['hour']-G23['hour'].iloc[0]# пока е0 минус t каждое
G23['t^2'] = G23['t']**2
G23['w'] = 1/G23['S'] *(  1+ ((2* G23['t'])/0.5)**2     )**-1 # ves =  delt t / interval
G23['(w)']=G23['w'].sum()
G23['(w*S)']=(G23['w']*G23['S']).sum()
G23['(w*S*lat)']=(G23['w']*G23['S']*G23['lat']).sum()
G23['(w*S*lat^2)']=(G23['w']*G23['S']*G23['lat^2']).sum()
G23['(w*S*lon)']=(G23['w']*G23['S']*G23['lon']).sum()
G23['(w*S*lon^2)']=(G23['w']*G23['S']*G23['lon^2']).sum()
G23['(w*S*t)']=  (G23['w']*G23['S']*G23['t']).sum()
G23['(w*S*t^2)']=(G23['w']*G23['S']*G23['t^2']).sum()
G23['(w*I)']=G23['w']*G23['vTEC'].sum()


G23['w*S^2']=(G23['w']*G23['S']**2)
G23['w*S^2*lat'] = (G23['w']*G23['S']**2*G23['lat'])
G23['w*S^2*lat^2']=(G23['w']*G23['S']**2*G23['lat']**2)
G23['w*S^2*lat^3']=(G23['w']*G23['S']**2*G23['lat']**3)
G23['w*S^2*lat^4']=(G23['w']*G23['S']**2*G23['lat']**4)
G23['w*S^2*lon'] = (G23['w']*G23['S']**2*G23['lon'])
G23['w*S^2*lon^2'] = (G23['w']*G23['S']**2*G23['lon']**2)
G23['w*S^2*lon^3'] = (G23['w']*G23['S']**2*G23['lon']**3)
G23['w*S^2*lon^4'] = (G23['w']*G23['S']**2*G23['lon']**4)
G23['w*S^2*lon']=(G23['w']*G23['S']**2*G23['lon'])
G23['w*S^2*t']=(G23['w']*G23['S']**2*G23['t'])
G23['w*S^2*t^2']=(G23['w']*G23['S']**2*G23['t']**2)
G23['w*S^2*t^3']=(G23['w']*G23['S']**2*G23['t']**3)
G23['w*S^2*t^4']=(G23['w']*G23['S']**2*G23['t']**4)
G23['w*S^2*lat*lon']=(G23['w']*G23['S']**2*G23['lat']*G23['lon'])
G23['w*S^2*lat^2*lon']=(G23['w']*G23['S']**2*G23['lat']**2*G23['lon'])
G23['w*S^2*lat*lon^2']=(G23['w']*G23['S']**2*G23['lat']*G23['lon']**2)
G23['w*S^2*lat^2*lon^2']=(G23['w']*G23['S']**2*G23['lat']**2*G23['lon']**2)
G23['w*S^2*lat*t']=(G23['w']*G23['S']**2*G23['lat']*G23['t'])
G23['w*S^2*lat^2*t']=(G23['w']*G23['S']**2*G23['lat']**2*G23['t'])
G23['w*S^2*lat*t^2']=(G23['w']*G23['S']**2*G23['lat']*G23['t']**2)
G23['w*S^2*lat^2*t^2']=(G23['w']*G23['S']**2*G23['lat']**2*G23['t']**2)
G23['w*S^2*lon*t']=  (G23['w']*G23['S']*G23['lon']*G23['t'])
G23['w*S^2*lon^2*t']=  (G23['w']*G23['S']**2*G23['lon']**2*G23['t'])
G23['w*S^2*lon*t^2']=  (G23['w']*G23['S']**2*G23['lon']*G23['t']**2)
G23['w*S^2*lon^2*t^2']=(G23['w']*G23['S']**2*G23['lon']**2*G23['t']**2)
G23['w*S*I'] = (G23['w']*G23['S']*G23['vTEC'])
G23['w*S*I*lat'] = (G23['w']*G23['S']*G23['vTEC']*G23['lat'])
G23['w*S*I*lat^2'] = (G23['w']*G23['S']*G23['vTEC']*G23['lat']**2)
G23['w*S*I*lon'] = (G23['w']*G23['S']*G23['vTEC']*  G23['lon'])
G23['w*S*I*lon^2'] = (G23['w']*G23['S']*G23['vTEC']*G23['lon']**2)
G23['w*S*I*t'] = (G23['w']*G23['S']*G23['vTEC']*  G23['t'])
G23['w*S*I*t^2'] = (G23['w']*G23['S']*G23['vTEC']*G23['t']**2)
#print(G23)
#G18.to_csv('proverka.csv',sep=';')
#G23.to_csv('G23.csv',sep=';')
#G23['(w*S*lat)'] = G23['w*S*lat'].sum()
#G23.plot.scatter(x='hour',y='vTEC',c='DarkBlue')
#G10.plot.scatter(x='hour',y='sTEC',c='DarkBlue')

GG5=   G5.loc[0,['(w)','(w*S)','(w*S*lat)','(w*S*lat^2)','(w*S*lon)','(w*S*lon^2)','(w*S*t)','(w*S*t^2)']]
GG10= G10.loc[0,['(w)','(w*S)','(w*S*lat)','(w*S*lat^2)','(w*S*lon)','(w*S*lon^2)','(w*S*t)','(w*S*t^2)']]
GG13= G13.loc[0,['(w)','(w*S)','(w*S*lat)','(w*S*lat^2)','(w*S*lon)','(w*S*lon^2)','(w*S*t)','(w*S*t^2)']]
GG15= G15.loc[0,['(w)','(w*S)','(w*S*lat)','(w*S*lat^2)','(w*S*lon)','(w*S*lon^2)','(w*S*t)','(w*S*t^2)']]
GG18= G18.loc[0,['(w)','(w*S)','(w*S*lat)','(w*S*lat^2)','(w*S*lon)','(w*S*lon^2)','(w*S*t)','(w*S*t^2)']]
GG23 =G23.loc[0,['(w)','(w*S)','(w*S*lat)','(w*S*lat^2)','(w*S*lon)','(w*S*lon^2)','(w*S*t)','(w*S*t^2)']]
res = pd.concat([GG5, GG10 ,GG13, GG15, GG18, GG23], axis=1)
res.to_csv('A2.csv',sep=';')

Gb5=   G5.loc[0,['(w*I)']]
Gb10= G10.loc[0,['(w*I)']]
Gb13= G13.loc[0,['(w*I)']]
Gb15= G15.loc[0,['(w*I)']]
Gb18= G18.loc[0,['(w*I)']]
Gb23 =G23.loc[0,['(w*I)']]
resb = pd.concat([Gb5, Gb10 ,Gb13, Gb15, Gb18, Gb23], axis=1)
resb_T = resb.T # or df1.tr'(w*S*I)',anspo*Ise()
resb_T.to_csv('b.csv',sep=';')
#resb_T
#print(G18,G23)
sum_J_A= (G5.loc[:,['w*S^2','w*S^2*lat','w*S^2*lat^2','w*S^2*lat^3','w*S^2*lat^4','w*S^2*lon','w*S^2*lon^2','w*S^2*lon^3','w*S^2*lon^4','w*S^2*lat*lon','w*S^2*lat^2*lon','w*S^2*lat*lon^2','w*S^2*lat^2*lon^2','w*S^2*t','w*S^2*t^2','w*S^2*t^3','w*S^2*t^4','w*S^2*lat*t','w*S^2*lat^2*t','w*S^2*lat*t^2','w*S^2*lat^2*t^2','w*S^2*lon*t','w*S^2*lon^2*t','w*S^2*lon*t^2','w*S^2*lon^2*t^2']]+
          G10.loc[:,['w*S^2','w*S^2*lat','w*S^2*lat^2','w*S^2*lat^3','w*S^2*lat^4','w*S^2*lon','w*S^2*lon^2','w*S^2*lon^3','w*S^2*lon^4','w*S^2*lat*lon','w*S^2*lat^2*lon','w*S^2*lat*lon^2','w*S^2*lat^2*lon^2','w*S^2*t','w*S^2*t^2','w*S^2*t^3','w*S^2*t^4','w*S^2*lat*t','w*S^2*lat^2*t','w*S^2*lat*t^2','w*S^2*lat^2*t^2','w*S^2*lon*t','w*S^2*lon^2*t','w*S^2*lon*t^2','w*S^2*lon^2*t^2']]+
          G13.loc[:,['w*S^2','w*S^2*lat','w*S^2*lat^2','w*S^2*lat^3','w*S^2*lat^4','w*S^2*lon','w*S^2*lon^2','w*S^2*lon^3','w*S^2*lon^4','w*S^2*lat*lon','w*S^2*lat^2*lon','w*S^2*lat*lon^2','w*S^2*lat^2*lon^2','w*S^2*t','w*S^2*t^2','w*S^2*t^3','w*S^2*t^4','w*S^2*lat*t','w*S^2*lat^2*t','w*S^2*lat*t^2','w*S^2*lat^2*t^2','w*S^2*lon*t','w*S^2*lon^2*t','w*S^2*lon*t^2','w*S^2*lon^2*t^2']]+
          G15.loc[:,['w*S^2','w*S^2*lat','w*S^2*lat^2','w*S^2*lat^3','w*S^2*lat^4','w*S^2*lon','w*S^2*lon^2','w*S^2*lon^3','w*S^2*lon^4','w*S^2*lat*lon','w*S^2*lat^2*lon','w*S^2*lat*lon^2','w*S^2*lat^2*lon^2','w*S^2*t','w*S^2*t^2','w*S^2*t^3','w*S^2*t^4','w*S^2*lat*t','w*S^2*lat^2*t','w*S^2*lat*t^2','w*S^2*lat^2*t^2','w*S^2*lon*t','w*S^2*lon^2*t','w*S^2*lon*t^2','w*S^2*lon^2*t^2']]+
          G18.loc[:,['w*S^2','w*S^2*lat','w*S^2*lat^2','w*S^2*lat^3','w*S^2*lat^4','w*S^2*lon','w*S^2*lon^2','w*S^2*lon^3','w*S^2*lon^4','w*S^2*lat*lon','w*S^2*lat^2*lon','w*S^2*lat*lon^2','w*S^2*lat^2*lon^2','w*S^2*t','w*S^2*t^2','w*S^2*t^3','w*S^2*t^4','w*S^2*lat*t','w*S^2*lat^2*t','w*S^2*lat*t^2','w*S^2*lat^2*t^2','w*S^2*lon*t','w*S^2*lon^2*t','w*S^2*lon*t^2','w*S^2*lon^2*t^2']]+
          G23.loc[:,['w*S^2','w*S^2*lat','w*S^2*lat^2','w*S^2*lat^3','w*S^2*lat^4','w*S^2*lon','w*S^2*lon^2','w*S^2*lon^3','w*S^2*lon^4','w*S^2*lat*lon','w*S^2*lat^2*lon','w*S^2*lat*lon^2','w*S^2*lat^2*lon^2','w*S^2*t','w*S^2*t^2','w*S^2*t^3','w*S^2*t^4','w*S^2*lat*t','w*S^2*lat^2*t','w*S^2*lat*t^2','w*S^2*lat^2*t^2','w*S^2*lon*t','w*S^2*lon^2*t','w*S^2*lon*t^2','w*S^2*lon^2*t^2']]).sum()
sum_J_b= (G5.loc[:,['w*S*I','w*S*I*lat','w*S*I*lat^2','w*S*I*lon','w*S*I*lon^2','w*S*I*t','w*S*I*t^2']]+
          G10.loc[:,['w*S*I','w*S*I*lat','w*S*I*lat^2','w*S*I*lon','w*S*I*lon^2','w*S*I*t','w*S*I*t^2']]+
          G13.loc[:,['w*S*I','w*S*I*lat','w*S*I*lat^2','w*S*I*lon','w*S*I*lon^2','w*S*I*t','w*S*I*t^2']]+
          G15.loc[:,['w*S*I','w*S*I*lat','w*S*I*lat^2','w*S*I*lon','w*S*I*lon^2','w*S*I*t','w*S*I*t^2']]+
          G18.loc[:,['w*S*I','w*S*I*lat','w*S*I*lat^2','w*S*I*lon','w*S*I*lon^2','w*S*I*t','w*S*I*t^2']]+
          G23.loc[:,['w*S*I','w*S*I*lat','w*S*I*lat^2','w*S*I*lon','w*S*I*lon^2','w*S*I*t','w*S*I*t^2']]).sum()
#print(G18.loc[:,['w','S','w*S^2']])
#print(G23.loc[:,['w','S','w*S^2']])
sum_J_A.to_csv('J_A.csv',sep=';')
sum_J_b.to_csv('J_b.csv',sep=';')

'ok'

ddddddd
0      1.646002
1      1.652145
2      1.658330
3      1.664557
4      1.670825
         ...   
109    2.561006
110    2.570980
111    2.580946
112    2.590902
113    2.600847
Name: S, Length: 114, dtype: float64
0     0.241667
1     0.241667
2     0.241667
3     0.241667
4     0.241667
5     0.241667
6     0.241667
7     0.241667
8     0.241667
9     0.241667
10    0.241667
11    0.241667
12    0.241667
13    0.241667
14    0.241667
15    0.241667
16    0.241667
17    0.241667
18    0.241667
19    0.241667
20    0.241667
21    0.241667
22    0.241667
23    0.241667
24    0.241667
25    0.241667
26    0.241667
27    0.241667
28    0.241667
29    0.241667
Name: EnBg, dtype: float64


KeyError: ' t'

In [333]:
matrixA = pd.read_csv ('a.csv',sep = ';')
matrixB = matrixA['b']
matrixA.drop(['b'], axis=1, inplace=True)

b = matrixB.values
A = matrixA.values

w = np.linalg.lstsq(A.T, b, rcond=None)[0]
w
x = pd.Series(w) 
x

0     8490.551987
1     -408.700229
2        4.612529
3        0.242795
4        0.031094
5       -0.123558
6     6178.057142
7      524.375198
8      528.326002
9      376.809563
10     467.793729
11     422.999699
12     461.997854
dtype: float64

NameError: name 'dd' is not defined

[9.59340274e+03 4.05896289e+05 1.72070669e+07 7.49334745e+05
 5.87524299e+07 7.47012736e+02 8.01826635e+01 7.78600907e+02
 6.03766416e+02 1.58229475e+03 1.10550689e+03 1.95344307e+03
 1.36138691e+03]


0     -3081.076254
1       250.695705
2       -17.672817
3        -0.095254
4         2.260665
5         0.203867
6      6712.325360
7     19522.949460
8     20581.866693
9      9606.704104
10    14441.691761
11    12503.287882
12    13756.954695
dtype: float64

[[1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1]]